In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import nbformat
from langchain_core.documents import Document
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

# 현재 노트북 파일의 상위 폴더(Root)를 경로에 추가
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from src.utils.config import ConfigDB

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
# Python TXT Docs 전용 설정
PYTHON_DOCS_ROOT = "../data/raw/python-3.14-docs-text"
PYTHON_DOC_INCLUDE_DIRS = ["library", "reference"]  # 최소 범위부터

def split_rst_sections(text: str):
    """
    reST 스타일 섹션 분리:
    제목 줄 + underline(====, ---- 등) 패턴 기준
    """
    lines = text.splitlines()
    sections = []
    current_title = "ROOT"
    buf = []

    def flush():
        nonlocal buf
        content = "\n".join(buf).strip()
        if content:
            sections.append((current_title, content))
        buf = []

    i = 0
    while i < len(lines):
        line = lines[i].rstrip()
        if i + 1 < len(lines):
            underline = lines[i + 1].rstrip()
            if underline and len(underline) >= max(3, len(line)):
                if len(set(underline)) == 1 and underline[0] in "= -~^*+_":
                    flush()
                    current_title = line.strip() or current_title
                    i += 2
                    continue
        buf.append(line)
        i += 1

    flush()
    return sections

def load_python_txt_documents(root_dir: str, include_dirs=None):
    documents = []
    for dirpath, _, filenames in os.walk(root_dir):
        for fn in filenames:
            if not fn.endswith(".txt"):
                continue

            full_path = os.path.join(dirpath, fn)
            rel_path = os.path.relpath(full_path, root_dir)
            top_dir = rel_path.split(os.sep)[0]
            if include_dirs and top_dir not in include_dirs:
                continue

            with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            for section, section_text in split_rst_sections(text):
                documents.append(
                    Document(
                        page_content=section_text,
                        metadata={
                            "source": "python_doc",
                            "title": rel_path.replace(os.sep, "/"),
                            "section": section,
                        },
                    )
                )
    return documents


In [4]:
# Python TXT Docs 로드
python_docs = load_python_txt_documents(
    root_dir=PYTHON_DOCS_ROOT,
    include_dirs=PYTHON_DOC_INCLUDE_DIRS,
)

print("Loaded python_docs:", len(python_docs))
assert len(python_docs) > 0, (
    "python_docs가 0개입니다. PYTHON_DOCS_ROOT / 압축해제 위치 / include_dirs 확인 필요"
)

print("Sample metadata:", python_docs[0].metadata)

Loaded python_docs: 2469
Sample metadata: {'source': 'python_doc', 'title': 'library/abc.txt', 'section': '"abc" --- 추상 베이스 클래스'}


In [5]:
assert len(python_docs) > 0, "python_docs가 비어있습니다."

chunk_size = 1000 # 너무 작은거같아서 키웠어요
chunk_overlap = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# 여기서 documents 대상 결정
documents = text_splitter.split_documents(python_docs)   # python docs를 chunk 대상으로

# 품질 샘플 확인(전부 print는 너무 많으니 3~5개만)
for d in documents[:5]:
    print(">>>>>", d.metadata)
    print(d.page_content[:400])
    print("-" * 80)

>>>>> {'source': 'python_doc', 'title': 'library/abc.txt', 'section': '"abc" --- 추상 베이스 클래스'}
**소스 코드:** Lib/abc.py
--------------------------------------------------------------------------------
>>>>> {'source': 'python_doc', 'title': 'library/abc.txt', 'section': '"abc" --- 추상 베이스 클래스'}
이 모듈은, **PEP 3119**에서 설명된 대로, 파이썬에서 *추상 베이스 클래
스* (ABC) 를 정의하기 위한 기반 구조를 제공합니다; 이것이 왜 파이썬에
추가되었는지는 PEP를 참조하십시오. (ABC를 기반으로 하는 숫자의 형 계층
구조에 관해서는 **PEP 3141**과 "numbers" 모듈을 참고하십시오.)

"collections" 모듈은 ABC로부터 파생된 몇 가지 구상(concrete) 클래스를
가지고 있습니다; 이것은 물론 더 파생될 수 있습니다. 또한,
"collections.abc" 서브 모듈에는 클래스나 인스턴스가 특정 인터페이스를
(예를 들어, *해시 가능*이거나 *매핑*이면) 제공하는지 검사하는 데 사용
할 수 있는 ABC가 있습니다.

이 모듈은 ABC를 정의하기 위한 메타 
--------------------------------------------------------------------------------
>>>>> {'source': 'python_doc', 'title': 'library/abc.txt', 'section': '"abc" --- 추상 베이스 클래스'}
class MyABC(metaclass=ABCMeta):
          pass

   Added in version 3.4.

class abc.ABCMeta

   추상 베이스 클래스 (ABC)를 정의하기 위한 메타 클래스.

  

In [6]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

client = QdrantClient(host="localhost", port=6333)

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name=ConfigDB.COLLECTION_NAME,
    embedding=embedding_model,
)

In [7]:
# Vector Store 생성

client = QdrantClient(host="localhost", port=6333)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = QdrantVectorStore(
    client=client,
    collection_name=ConfigDB.COLLECTION_NAME,
    embedding=embedding_model
)

ids = vector_store.add_documents(documents)
ids

['964d952c21084111985e391c3d0b1381',
 '85ef51ed8c0b42faa96edb6631c6aabc',
 '3d3e4e8937f24661877d7fc4f1b4dd36',
 'cc6e3ecc344f464c8e9fe943f3db79f3',
 '1f8ab33471a34aa29fb334f69a8e1882',
 '8db373393b98499ab37a32b9059972e1',
 '16229a76f87e4410aef563cc43fb3c48',
 '2af4344d8cd24f4b9034a7e6131eda42',
 'daeeb7dbcde842c39f8b4e9fb61df4cb',
 'a0a17d927eb34ebc8676886acdb5a2e0',
 '1f99f24d8f3c4d33a4b24a3a8246590c',
 '7098ca7169294a7098c1cc3b5939d3c1',
 'cc16b48396ef45c2aa11a1f2ea94be1e',
 '1de2e89d93c040e99d8aaabbac2200c4',
 '1e562e2de8af42869e1fe21dd0ed01e3',
 '7432af8eed3145b18cfd96b8481d0656',
 'adf7ae13c71e4477828cec0450511533',
 '70a352142c5f45d9b75d16829c044cdd',
 'c89e5cb3b21c4e368b4e6510e85b77cb',
 '0c735a6fe7f341a5b6d608f3db7ec83a',
 '63bd82b590884e538f2254507199fbeb',
 'd7a0cf9e4a424a79bc551bac1ac27f89',
 '0cfc8df371594e2e8310b330fb145c8c',
 '36d3a4fedd4a4a1c9b87405efe17ad41',
 '885dd253e0ca4d14b1675c550166adf1',
 'c8eb0ea95c8343a79dc49d0460d817e8',
 'f92bb13a7f394b7fbe491fb34f170a83',
 

In [8]:
# 테스트용 질문입니당당

questions = [
    "pathlib.Path는 무엇을 위한 클래스야?",
    "json.dump와 json.dumps의 차이는?",
    "open() 함수의 encoding 파라미터는 무엇을 의미하니?",
    "asyncio.run을 중첩 호출하면 안 되는 이유는?",
    "GIL이 파이썬 성능에 미치는 영향은?",
]

for q in questions:
    print("\n" + "="*20)
    print("Q:", q)
    results = vector_store.similarity_search(q, k=3)
    for i, r in enumerate(results, 1):
        print(f"  [{i}] {r.metadata.get('title')} | {r.metadata.get('section')}")
        print("      ", r.page_content[:200].replace("\n", " "))


Q: pathlib.Path는 무엇을 위한 클래스야?
  [1] library/pathlib.txt | "pathlib" --- Object-oriented filesystem paths
       이 모듈은 다른 운영 체제에 적합한 의미 체계를 가진 파일 시스템 경로를 나타내는 클래스를 제공합니다. 경로 클래스는 I/O 없이 순수한 계산 연산 을 제공하는 순수한 경로와 순수한 경로를 상속하지만, I/O 연산도 제공하 는 구상 경로로 구분됩니다.  [그림: Inheritance diagram showing the classes available in 
  [2] library/pathlib.txt | "pathlib" --- Object-oriented filesystem paths
       이 모듈은 다른 운영 체제에 적합한 의미 체계를 가진 파일 시스템 경로를 나타내는 클래스를 제공합니다. 경로 클래스는 I/O 없이 순수한 계산 연산 을 제공하는 순수한 경로와 순수한 경로를 상속하지만, I/O 연산도 제공하 는 구상 경로로 구분됩니다.  [그림: Inheritance diagram showing the classes available in 
  [3] library/pathlib.txt | "pathlib" --- Object-oriented filesystem paths
       이 모듈은 다른 운영 체제에 적합한 의미 체계를 가진 파일 시스템 경로를 나타내는 클래스를 제공합니다. 경로 클래스는 I/O 없이 순수한 계산 연산 을 제공하는 순수한 경로와 순수한 경로를 상속하지만, I/O 연산도 제공하 는 구상 경로로 구분됩니다.  [그림: Inheritance diagram showing the classes available in 

Q: json.dump와 json.dumps의 차이는?
  [1] library/json.txt | 기본 사용법
       * **default** (*callable* | None) -- 달리 직렬화할 수 없는